# Mini-projet d'optimisation : Effacement de consommation

## 1. Etude du problème d'optimisation

1. La facture d'énergie s'écrit comme l'intégrale de la puissance électrique consommée multipliée par le prix de l'énergie sur une période de temps donnée.
 En discrétisant le temps, on peut écrire la facture d'énergie comme la somme des puissances électriques consommées multipliée par le prix de l'énergie sur chaque intervalle de temps.
 Ainsi : 
 $$\text{Facture} = \Delta t \sum_{i=0}^{N-1} c_i P_i$$

2. Le bilan énergétique de la maison s'écrit :
- $P$ la puissance thermique produite par les convecteurs.
- $C \frac{dT}{dt}$ la puissance thermique absorbée par les murs et l'air dans la maison (on considère un système dynamique, la maison étant de capacité calorifique $C$)
- $H(T-T^e)$ la puissance thermique dissippée à l'exérieur par les murs et les fenêtres, avec le coefficient d'échange conducto-convectif $H$.
- $KT$ un facteur de conduction vers le sol (on considère alors qu'il est à température nulle, ce qui est une hypothèse forte).

Ainsi, le bilan énergétique s'écrit :
$$P = C \frac{dT}{dt} + H(T-T^e) + KT$$

La solution de ce problème s'écrit :
$$T(t) = \frac{H}{H+K}T^e + \frac{P}{H+K} + (T_0 - \frac{H}{H+K}T^e - \frac{P}{H+K})e^{-\frac{H+K}{C}t}$$

Ce qui se discrétise en :
$$T_{i+1} = \frac{H}{H+K}T^e_i + \frac{P_i}{H+K} + (T_i - \frac{H}{H+K}T^e_i - \frac{P_i}{H+K})e^{-\frac{H+K}{C}\Delta t}$$



On identifie alors $h$, $k$ et $b$, pour obtenir :
$$T_{i+1} = e^{-(k+h)\Delta t} T_i + \frac{1-e^{-(k+h)\Delta t}}{k+h} (bP_i + hT_i^e)$$
avec : 
$$h = \frac{H}{C} ; k = \frac{K}{C} ; b = \frac{1}{C}$$

3. On veut minimiser la facture . Donc le problème d'optimisation est $$\min_{P_i} \Delta t \sum_{i=0}^{N-1} c_i P_i$$

Les variables de décision sont donc les $P_i$, au nombre de $n=N$. 

On a également les contraintes inégalités suivantes : 
$$(c_{I1})_i = T_m-T_i$$
$$(c_{I2})_i = T_i-T_M$$
$$(c_{I3})_i = -P_i$$
$$(c_{I4})_i = P_i-P_M$$

La fonction $f$ est définie par :
$$f(P) = \Delta t \sum_{i=0}^{N-1} c_i P_i$$

4. Les $c_i$ sont constants. Donc la fonction $f$ est linéaire en $P$. Le problème est donc un problème d'optimisation linéaire.

## 2. Etude et résolution numérique du problème individuel

5. La fonction $f$ est différentiable, donc la résolution de ce problème aux contraintes peut se faire par un algorithme de recherche de point selle, comme l'algorithme d'Uzawa ou d'Arrow-Hurwitz. Etant donné que le problème est linéaire, on choisira une méthode à contraites actives de la bibliothèque scipy. L'algorithme du simplexe à convergence garantie pourrait être mise en oeuvre pour le résoudre à la main.

## 3. Régulation collective

8. La fonction f à minimiser est désormais : 

$$ f(P) = \Delta t \sum_{i=0}^{N-1} c_i \Biggl( \sum_{j=0}^{n_l-1}P_i^j \Biggr) $$

Où $P$, le vecteur des  $P_i^j$, constitue la variable de décision. Les $P_i^j$ sont au nombre de $N \cdot n_l$.

Cette fois, on a les contraintes inégalités suivantes, pour tous $i, j \in [\![0, N-1]\!]  \times [\![0, n_l-1]\!]$ :

$$ (c_{I1})_{i,j} = T_m^j-T_i^j $$
$$ (c_{I2})_{i,j} = T_i^j - T_M^j $$
$$ (c_{I3})_{i,j} = -P_i^j $$
$$ (c_{I4})_{i,j} = P_i^j - P_M $$

Le problème s'écrit donc : $\min_{P} f(P)$, où $P$ est un vecteur de taille $N \cdot n_l$.

In [ ]:
#10. Mise en oeuvre de la méthode du simplexe pour n_l = 2

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import 

#Définition de la fonction de température
def T()
